* **reference:** https://python-client.qdrant.tech/

In [ ]:
## Qdrant
from qdrant_client import QdrantClient
from dotenv import load_dotenv
import os

load_dotenv()
client = QdrantClient(
    url=os.getenv("QDRANT_URL"), 
    api_key=os.getenv("QDRANT_API_KEY"),
)

print(client.get_collections())

/home/anon-labs/Documents/Hacker-House-1/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


collections=[CollectionDescription(name='my_collection'), CollectionDescription(name='test')]


## Embedding and inserting

In [2]:
# Prepare your documents, metadata, and IDs
docs = ["Qdrant has Langchain integrations", "Qdrant also has Llama Index integrations"]

client.add(
    collection_name="demo_collection",
    documents=docs,
)

Fetching 5 files: 100%|██████████| 5/5 [00:04<00:00,  1.18it/s]


['d4a1f4b94c9848d0a2ac13121fd29276', '76ac7725a56d4d0b9433670e0f633502']

In [3]:
# If you'd like to add points with control over the metadata, you can use the metadata parameter. Here is a quick example:

# Prepare your documents, metadata, and IDs
docs = ["Qdrant has Langchain integrations", "Qdrant also has Llama Index integrations"]
metadata = [
    {"source": "Langchain-docs"},
    {"source": "Linkedin-docs"},
]
ids = [42, 2]

# Use the new add method
client.add(
    collection_name="demo_collection",
    documents=docs,
    metadata=metadata,
    ids=ids
)

[42, 2]

## Querying with text directly
At query time, we need to embed the incoming query and then search for the nearest neighbors. We can do this with the query API:


In [4]:
search_result = client.query(
    collection_name="demo_collection",
    query_text="This is a query document",
    limit=1
)
print(search_result)


[QueryResponse(id='d4a1f4b9-4c98-48d0-a2ac-13121fd29276', embedding=None, sparse_embedding=None, metadata={'document': 'Qdrant has Langchain integrations'}, document='Qdrant has Langchain integrations', score=0.8133327)]


## Qdrant without fastembed
- Collection
- A collection is a set of points with the same dimensionality and a similarity metric (e.g. Dot, Cosine) defined on it. We can create a collection with the create_collection method:


from qdrant_client.http.models import Distance, VectorParams

if not client.collection_exists("test_collection"):
    client.create_collection(
            collection_name="test_collection",
            vectors_config=VectorParams(size=4, distance=Distance.DOT),
    )

In [5]:
from qdrant_client.http.models import Distance, VectorParams

if not client.collection_exists("test_collection"):
    client.create_collection(
            collection_name="test_collection",
            vectors_config=VectorParams(size=4, distance=Distance.DOT),
    )

### Points
A point is a vector of floats with a unique identifier id. We can create a point with the Point API:

In [6]:

from qdrant_client.http.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ]
)
print(operation_info)
# operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>
# This creation of Points is also abstracted away with the add API.

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


## Querying with vector

In [8]:
search_result = client.search(
    collection_name="test_collection",
    query_vector=[0.18, 0.81, 0.75, 0.12],
    limit=1
)
print(search_result)

/tmp/ipykernel_8459/2965081881.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(


[ScoredPoint(id=2, version=0, score=1.266, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]


## Querying with a Filter and Vector


In [9]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue

search_result = client.search(
    collection_name="test_collection",
    query_vector=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[
            FieldCondition(
                key="city",
                match=MatchValue(value="London")
            )
        ]
    ),
    limit=1
)
print(search_result)

/tmp/ipykernel_8459/2165647533.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(


[ScoredPoint(id=2, version=0, score=0.871, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]


## References:
* [Docs](https://python-client.qdrant.tech/)

In [8]:
%pip install qdrant-client --quiet
%pip install python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
